# 🧠 MNIST Digit Classification

In this notebook, we’ll build, train, and evaluate two models for classifying handwritten digits from the **MNIST dataset**:

1. A simple **Artificial Neural Network (ANN)**.
2. A **Convolutional Neural Network (CNN)** for improved performance on image data.

We’ll also try out real handwritten digit images to see how well our models generalize beyond the MNIST dataset.

## 📦 Step 1: Load the MNIST Dataset

We’ll use TensorFlow’s built-in dataset loader. The MNIST dataset contains **60,000 training** and **10,000 test** grayscale images of handwritten digits (0–9), each of size 28×28 pixels.

In [ ]:
from tensorflow.keras.datasets import mnist

# Load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(f"Training samples: {x_train.shape[0]}, Test samples: {x_test.shape[0]}")

## 🔧 Step 2: Preprocess the Data

- Normalize pixel values from 0–255 to 0–1 for better numerical stability.
- Flatten 28×28 images into 784-dimensional vectors for the ANN.
- One-hot encode the labels (e.g., digit 3 → `[0,0,0,1,0,0,0,0,0,0]`).

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np

# Normalize pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flatten images (28x28 → 784)
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")

## 🏗️ Step 3: Build the Neural Network (ANN)

We’ll start with a simple feedforward neural network with:
- 2 hidden layers (128 and 64 neurons, ReLU activation)
- Output layer (10 neurons, softmax for class probabilities)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 🧮 Step 4: Train the Model

We’ll train for 10 epochs and observe both training and validation accuracy to check for overfitting.

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

## 📊 Step 5: Evaluate Model Performance

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

## 📈 Step 6: Visualize Training History

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

## 🖼️ Step 7: Test on Custom Handwritten Digits

You can upload images named `0.jpg`, `1.jpg`, … `9.jpg` and test how the model performs on your own handwriting.

We'll preprocess each image just like MNIST: grayscale → invert → normalize → flatten.

In [ ]:
import glob, os
from PIL import Image
import numpy as np

correct, total = 0, 0
plt.figure(figsize=(12, 6))

for idx, file_name in enumerate(sorted(glob.glob('*.jpg'))):
    img = Image.open(file_name).convert('L').resize((28, 28))
    img_array = np.array(img)
    if img_array.mean() > 127:
        img_array = 255 - img_array
    img_array = img_array / 255.0
    img_array_flat = img_array.reshape(1, 784)

    prediction = model.predict(img_array_flat, verbose=0)
    pred_digit = np.argmax(prediction)
    actual_digit = int(os.path.splitext(file_name)[0])

    if pred_digit == actual_digit:
        correct += 1
    total += 1

    plt.subplot(2, 5, idx + 1)
    plt.imshow(img_array, cmap='gray')
    plt.title(f'Pred: {pred_digit}\nAct: {actual_digit}')
    plt.axis('off')

acc = (correct / total * 100) if total else 0
print(f"Custom image accuracy: {acc:.2f}%")
plt.tight_layout()
plt.show()

## 🧱 Step 8: Build a CNN for Better Image Understanding

A **Convolutional Neural Network (CNN)** leverages spatial structure in images, leading to significantly higher accuracy.

**Key improvements over ANN:**
- Local connectivity (focuses on small image patches)
- Weight sharing (same filter detects features anywhere)
- Translation invariance (digit position doesn’t matter)

In [ ]:
from tensorflow.keras import datasets, layers, models

# Reload MNIST (for 2D image input)
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

# Build CNN
cnn = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn.summary()

In [ ]:
# Train CNN
cnn_history = cnn.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [ ]:
# Evaluate CNN
test_loss, test_acc = cnn.evaluate(x_test, y_test)
print(f"\n✅ CNN Test Accuracy: {test_acc:.4f}")

## 💾 Step 9: Save Models for Future Use

In [ ]:
model.save('mnist_ann_model.h5')
cnn.save('mnist_cnn_model.h5')
print("Models saved: mnist_ann_model.h5 and mnist_cnn_model.h5")

## 🧩 Step 10: Summary

- ANN achieved decent performance but struggles with generalizing to non-MNIST digits.
- CNN captured spatial patterns better and achieved much higher accuracy.
- Real-world performance depends heavily on preprocessing quality (contrast, alignment, noise).

**Next Steps:**
- Add dropout or batch normalization.
- Train longer with data augmentation.
- Try digit segmentation for multiple-digit images.